# Deploy ML Model
https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/using_tf.html
https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/using_tf.html#deploy-tensorflow-serving-models


After a TensorFlow estimator has been fit, it saves a TensorFlow SavedModel in the S3 location defined by output_path. You can call deploy on a TensorFlow estimator to create a SageMaker Endpoint, or you can call transformer to create a Transformer that you can use to run a batch transform job.

In [ ]:
from sagemaker.tensorflow import TensorFlowModel

model = TensorFlowModel(model_data='s3://mybucket/model.tar.gz', role='MySageMakerRole')

predictor = model.deploy(initial_instance_count=1, instance_type='ml.c5.xlarge')

## Batch Transformation

From https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/using_tf.html#run-a-batch-transform-job 

and https://sagemaker.readthedocs.io/en/stable/overview.html#sagemaker-batch-transform

Create transformer object and run over an in input directory

In [ ]:
bucket = myBucket # The name of the S3 bucket where the results are stored
prefix = 'batch-results' # The folder in the S3 bucket where the results are stored

batch_output = 's3://{}/{}/results'.format(bucket, prefix) # The location to store the results

tf_transformer = tf_estimator.transformer(instance_count=1, instance_type='ml.m4.xlarge', output_path=batch_output)

tf_transformer.transform(data=batch_input, data_type='S3Prefix', content_type='text/csv', split_type='Line')